In [ ]:
import pandas as pd
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Caminho da pasta onde estão seus arquivos
caminho_base = r"C:\Users\lucas.lopes\Desktop\Relatórios de Apoio - CX"

# === 1. LER OS ARQUIVOS ===
estoque_df = pd.read_excel(os.path.join(caminho_base, "Desafio 2 Relatório de Estoque.xlsx"))
clientes_df = pd.read_excel(os.path.join(caminho_base, "Desafio 2 Base de Clientes.xlsx"))
pedidos_df = pd.read_excel(os.path.join(caminho_base, "Desafio 2 Base de Dados - Detalhes dos Pedidos .xlsx"))

# === 2. IDENTIFICAR PEDIDOS TRAVADOS ===
produtos_travados = estoque_df[estoque_df['Estoque'] == 0]
pedidos_travados = pedidos_df.merge(
    produtos_travados,
    left_on="Detalhe do pedido",
    right_on="Produtos",
    how="inner"
)
pedidos_travados_completo = pedidos_travados.merge(
    clientes_df,
    left_on="Pedido",
    right_on="Código do Pedido",
    how="left"
)

# === 3. PRIORIDADE POR DATA DO PEDIDO ===
pedidos_travados_completo["Data do Pedido"] = pd.to_datetime(pedidos_travados_completo["Data do Pedido"])
data_mais_antiga = pedidos_travados_completo["Data do Pedido"].min()
pedidos_travados_completo["Prioridade"] = pedidos_travados_completo["Data do Pedido"].apply(
    lambda x: "Mais Urgente" if x == data_mais_antiga else "Normal"
)

# === 4. BASE FINAL ===
resultado_final = pedidos_travados_completo[[
    "Pedido", "Detalhe do pedido", "Nome do Cliente",
    "Telefone", "E-mail", "Data do Pedido", "Prioridade"
]]
output_path = os.path.join(caminho_base, "Pedidos Travados com Prioridade.xlsx")
resultado_final.to_excel(output_path, index=False)

print(f"\n✅ Arquivo gerado com sucesso: {output_path}\n")
print("📋 Pedidos travados encontrados:\n")
print(resultado_final)

# === 5. ENVIO DE E-MAIL ===
remetente = "lucaswilkinsonlopes@gmail.com"
senha = ""  

def enviar_email(row):
    destinatario = row["E-mail"]
    nome = row["Nome do Cliente"]
    pedido = row["Pedido"]

    html = f"""
    <html>
    <body style="font-family: Arial, sans-serif;">
        <h2>Olá, {nome}!</h2>
        <p>Identificamos que um item do seu pedido <strong>{pedido}</strong> está temporariamente indisponível.</p>
        <p>Você pode escolher como deseja prosseguir clicando em uma das opções abaixo:</p>

        <a href="https://www.gocase.com.br/" 
           style="display:inline-block; padding:10px 20px; margin:5px; background:#4CAF50; color:white; text-decoration:none; border-radius:5px;">
           🕓 Aguardar Reposição
        </a>
        <a href="https://www.gocase.com.br/" 
           style="display:inline-block; padding:10px 20px; margin:5px; background:#2196F3; color:white; text-decoration:none; border-radius:5px;">
           🔁 Trocar Produto
        </a>
        <a href="https://www.gocase.com.br/" 
           style="display:inline-block; padding:10px 20px; margin:5px; background:#f44336; color:white; text-decoration:none; border-radius:5px;">
           ❌ Cancelar Pedido
        </a>

        <p style="margin-top:20px;">Qualquer dúvida, estamos à disposição! 💬</p>
    </body>
    </html>
    """

    msg = MIMEMultipart("alternative")
    msg["Subject"] = f"Ação necessária sobre o pedido {pedido}"
    msg["From"] = remetente
    msg["To"] = destinatario
    msg.attach(MIMEText(html, "html"))

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.login(remetente, senha)
        server.sendmail(remetente, destinatario, msg.as_string())
        server.quit()
        print(f"✅ E-mail enviado para {nome} ({destinatario})")
    except Exception as e:
        print(f"❌ Erro ao enviar para {destinatario}: {e}")

# Enviar para os 5 primeiros como teste
for _, row in resultado_final.head(5).iterrows():
    enviar_email(row)


❌ Erro ao enviar para lucaswpowerbi@gmail.com: [WinError 10053] Uma conexão estabelecida foi anulada pelo software no computador host
❌ Erro ao enviar para lucaswpowerbi@gmail.com: [WinError 10053] Uma conexão estabelecida foi anulada pelo software no computador host
❌ Erro ao enviar para lucaswpowerbi@gmail.com: [WinError 10053] Uma conexão estabelecida foi anulada pelo software no computador host
❌ Erro ao enviar para lucaswpowerbi@gmail.com: [WinError 10053] Uma conexão estabelecida foi anulada pelo software no computador host
❌ Erro ao enviar para lucaswpowerbi@gmail.com: [WinError 10053] Uma conexão estabelecida foi anulada pelo software no computador host
